In [1]:
import os
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from chess import pgn
from tqdm import tqdm
from auxilary_function import create_input_for_nn,encode_moves
from dataset import ChessDataset
from model import ChessModel

In [ ]:
def train(x,y,num_classes):

    dataset=ChessDataset(x,y)
    dataLoader=DataLoader(dataset,batch_size=64,shuffle=True)

    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using devices: {device}")

    global model
    if(model is None):
        model= ChessModel(num_classes=num_classes).to(device)
        print("\n-->model created<--\n")
    criterion=nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=0.0001)

    num_epochs=35
    for epoch in range(num_epochs):
        start_time=time.time()
        model.train()
        running_loss=0.0
        for inputs,labels in tqdm(dataLoader):
            inputs,labels=inputs.to(device),labels.to(device)
            optimizer.zero_grad()

            outputs=model(inputs)
            loss=criterion(outputs,labels)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)
            optimizer.step()
            running_loss+=loss.item()
        end_time=time.time()
        epoch_time=end_time-start_time
        minutes: int = int(epoch_time// 60)
        seconds: int = int(epoch_time)-minutes*60
        print(f'Epoch {epoch+1+50}/{num_epochs+1+50}, Loss: {running_loss/len(dataLoader):.4f}, Time: {minutes},{seconds}s')
        torch.save(model.state_dict(),"../models/epochgame60000.pth")

In [3]:
def doit(games):
    print(len(games))

    x,y=create_input_for_nn(games)
    print(f"number of samples: {len(y)}")

    x=x[0:2500000]
    y=y[0:2500000]

    y,move_to_int=encode_moves(y)
    num_classes=len(move_to_int)

    x=torch.tensor(x,dtype=torch.float32)
    y=torch.tensor(y,dtype=torch.long)

    train(x,y,num_classes)




    



In [4]:
def load_pgn(file_path,total_games,maxg):
    games=[]
    i=0
    with open(file_path, 'r' ) as pgn_file:
        while True:
            game=pgn.read_game(pgn_file)
            if game is None or i+total_games>=maxg:
                break
                
            games.append(game)
            i+=1
    print(f"{i}---{file_path}")
    return games




model =None
files = [file for file in os.listdir("../Data/pgn") if file.endswith(".pgn")]
files.sort()
games = []
i = 0
mi=0
game_cap = 60000
make_at_max=10000

for file in tqdm(files):
    li = load_pgn(f"../Data/pgn/{file}", len(games),make_at_max)
    games.extend(li)
    mi+=len(li)

    # Call global_move_dict when reaching each batch or at the end
    if mi>=make_at_max:

        doit(games)
        games = []
      
        i+=mi
        mi=0
          # free memory

    if i >= game_cap:
        
        break

    print(f"Total games --> {i}")
        

  0%|          | 0/79 [00:00<?, ?it/s]

10---../Data/pgn/lichess_elite_2013-09.pgn
Total games --> 0
4---../Data/pgn/lichess_elite_2013-11.pgn
Total games --> 0
3---../Data/pgn/lichess_elite_2014-01.pgn
Total games --> 0
23---../Data/pgn/lichess_elite_2014-02.pgn
Total games --> 0
5---../Data/pgn/lichess_elite_2014-03.pgn
Total games --> 0
7---../Data/pgn/lichess_elite_2014-04.pgn
Total games --> 0


  9%|▉         | 7/79 [00:00<00:04, 14.54it/s]

264---../Data/pgn/lichess_elite_2014-05.pgn
Total games --> 0
140---../Data/pgn/lichess_elite_2014-06.pgn
Total games --> 0


 13%|█▎        | 10/79 [00:01<00:08,  7.91it/s]

...

Epoch 83/86, Loss: 1.5672, Time: 0,53s


100%|██████████| 12369/12369 [00:52<00:00, 234.54it/s]


Epoch 84/86, Loss: 1.5490, Time: 0,52s


 38%|███▊      | 30/79 [3:11:22<5:12:34, 382.75s/it]  

Epoch 85/86, Loss: 1.5308, Time: 0,52s
